# Attempt based on Google's GraphCast

Their model takes input of weather, and predicts the weather 6 hours later

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re

data_dir = '../metdata'

files = os.listdir(data_dir)
files = [os.path.join(data_dir,x) for x in files]
files

pattern = r"([-]?\d+[\.]?\d+)N_([-]?\d+[\.]?\d+)E\.csv"
parsed_latlon = []
for file in files:
    match = re.search(pattern, file)
    if match:
        latitude = match.group(1)
        longitude = match.group(2)
    parsed_latlon.append((float(latitude), float(longitude)))
lat, lon = zip(*parsed_latlon)

kelvin_to_cels = -273.15



In [18]:
def readfile(path, index):
    df = pd.read_csv(path)
    headings = ["year", "month", "day", "hour", "temperature(celsius)", "precipitation", "u-wind", "v-wind"]
    df.columns = headings
    df["temperature(celsius)"] = df["temperature(celsius)"] + kelvin_to_cels
    df["wind-speed"] = np.sqrt(df["u-wind"]**2 + df["v-wind"]**2)
    df["longitude"] = lon[index]
    df["latitude"] = lat[index]
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    # df['week_num'] = df['datetime'].dt.isocalendar().week
    df['week_num'] = np.floor((df['datetime'] - df['datetime'].iloc[0]).dt.total_seconds() / (60 * 60 * 24 * 7))
    # df['fortnight'] = np.floor(df['week_num'] / 2)
    # df['day_num'] = np.floor((df['datetime'] - df['datetime'].iloc[0]).dt.total_seconds() / (60 * 60 * 24))
    df['day_num'] = df['datetime'].dt.dayofyear
    # df['X'] = df['week_num'] % 2 == 0
    df['hour_sin'] = np.sin(df['hour'] * (2. * np.pi / 24))
    df['hour_cos'] = np.cos(df['hour'] * (2. * np.pi / 24))
    min_year = np.min(df['year'])
    max_year = np.max(df['year'])
    df['year'] = (df["year"] - min_year) / (max_year - min_year)
    df['day_of_year_sin'] = np.sin(df['day_num'] * (2. * np.pi / 365))
    df['day_of_year_cos'] = np.cos(df['day_num'] * (2. * np.pi / 365))
    return df

In [19]:
def join_datasets(files):
    dfs = []
    for i, file in enumerate(files):
        df = readfile(file, i)
        dfs.append(df)
    dfs = pd.concat(dfs, axis = 0)
    dfs.reset_index(drop=True, inplace=True)
    return dfs

dfs = join_datasets([files[0]])
dfs_indices = np.random.choice(dfs.shape[0], int(dfs.shape[0]*0.2))
# dfs = dfs.sample(int(dfs.shape[0]//10))
dfs

,year,month,day,hour,temperature(celsius),precipitation,u-wind,v-wind,wind-speed,longitude,latitude,datetime,week_num,day_num,hour_sin,hour_cos,day_of_year_sin,day_of_year_cos
0,0.0,1,1,1,-2.00,0.000000e+00,1.72010,-1.6487,2.382636,286.01,40.75,1980-01-01 01:00:00,0.0,1,0.258819,0.965926,1.721336e-02,0.999852
1,0.0,1,1,2,-3.08,0.000000e+00,1.72320,-1.3887,2.213121,286.01,40.75,1980-01-01 02:00:00,0.0,1,0.500000,0.866025,1.721336e-02,0.999852
2,0.0,1,1,3,-3.60,0.000000e+00,1.74940,-1.3237,2.193760,286.01,40.75,1980-01-01 03:00:00,0.0,1,0.707107,0.707107,1.721336e-02,0.999852
3,0.0,1,1,4,-3.40,0.000000e+00,1.78490,-1.5519,2.365219,286.01,40.75,1980-01-01 04:00:00,0.0,1,0.866025,0.500000,1.721336e-02,0.999852
4,0.0,1,1,5,-4.11,0.000000e+00,1.76480,-1.8720,2.572723,286.01,40.75,1980-01-01 05:00:00,0.0,1,0.965926,0.258819,1.721336e-02,0.999852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341874,1.0,12,31,19,3.10,8.887000e-07,0.27815,3.0868,3.099307,286.01,40.75,2018-12-31 19:00:00,2034.0,365,-0.965926,0.258819,-2.449294e-16,1.000000
341875,1.0,12,31,20,3.45,3.548400e-05,-0.27715,3.2876,3.299261,286.01,40.75,2018-12-31 20:00:00,2034.0,365,-0.866025,0.500000,-2.449294e-16,1.000000
341876,1.0,12,31,21,3.26,1.339300e-04,-0.70951,3.6037,3.672881,286.01,40.75,2018-12-31 21:00:00,2034.0,365,-0.707107,0.707107,-2.449294e-16,1.000000
341877,1.0,12,31,22,2.38,2.838000e-04,-1.23100,2.8776,3.129847,286.01,40.75,2018-12-31 22:00:00,2034.0,365,-0.500000,0.866025,-2.449294e-16,1.000000


In [23]:
# all_data = [X_tr, y_tr, X_v, y_v, X_test, y_test]
all_data = [dfs]
vars_to_drop = ["datetime","month", "day", "u-wind","v-wind", "week_num", "longitude", "latitude", "day_num", "hour"]
for i in range(len(all_data)):
    for var in vars_to_drop:
        try:
            all_data[i] = all_data[i].drop(var, axis=1)
        except:
            pass


In [24]:
dfs = all_data[0]
dfs

,year,temperature(celsius),precipitation,wind-speed,hour_sin,hour_cos,day_of_year_sin,day_of_year_cos
0,0.0,-2.00,0.000000e+00,2.382636,0.258819,0.965926,1.721336e-02,0.999852
1,0.0,-3.08,0.000000e+00,2.213121,0.500000,0.866025,1.721336e-02,0.999852
2,0.0,-3.60,0.000000e+00,2.193760,0.707107,0.707107,1.721336e-02,0.999852
3,0.0,-3.40,0.000000e+00,2.365219,0.866025,0.500000,1.721336e-02,0.999852
4,0.0,-4.11,0.000000e+00,2.572723,0.965926,0.258819,1.721336e-02,0.999852
...,...,...,...,...,...,...,...,...
341874,1.0,3.10,8.887000e-07,3.099307,-0.965926,0.258819,-2.449294e-16,1.000000
341875,1.0,3.45,3.548400e-05,3.299261,-0.866025,0.500000,-2.449294e-16,1.000000
341876,1.0,3.26,1.339300e-04,3.672881,-0.707107,0.707107,-2.449294e-16,1.000000
341877,1.0,2.38,2.838000e-04,3.129847,-0.500000,0.866025,-2.449294e-16,1.000000


In [48]:
target = "temperature(celsius)"
data = dfs.copy()
data[f"target_{target}"] = dfs[target].shift(-6)
data = dfs.dropna()
data

,year,temperature(celsius),precipitation,wind-speed,hour_sin,hour_cos,day_of_year_sin,day_of_year_cos,target_temperature(celsius)
0,0.0,-2.00,0.0,2.382636,0.258819,0.965926,1.721336e-02,0.999852,-4.53
1,0.0,-3.08,0.0,2.213121,0.500000,0.866025,1.721336e-02,0.999852,-4.45
2,0.0,-3.60,0.0,2.193760,0.707107,0.707107,1.721336e-02,0.999852,-4.90
3,0.0,-3.40,0.0,2.365219,0.866025,0.500000,1.721336e-02,0.999852,-6.09
4,0.0,-4.11,0.0,2.572723,0.965926,0.258819,1.721336e-02,0.999852,-6.28
...,...,...,...,...,...,...,...,...,...
341868,1.0,-3.82,0.0,2.376264,-0.258819,-0.965926,-2.449294e-16,1.000000,3.10
341869,1.0,-1.61,0.0,1.936727,-0.500000,-0.866025,-2.449294e-16,1.000000,3.45
341870,1.0,0.48,0.0,3.036879,-0.707107,-0.707107,-2.449294e-16,1.000000,3.26
341871,1.0,1.63,0.0,3.155069,-0.866025,-0.500000,-2.449294e-16,1.000000,2.38


In [49]:
X = data.drop(columns=[f'target_{target}'])  # Features DataFrame
y = data[f'target_{target}']  # Target DataFrame

In [56]:
from sklearn.model_selection import train_test_split

X_train_temp, X_t, y_train_temp, y_t = train_test_split(X, y, test_size=0.1, random_state=42)

X_tr, X_v, y_tr, y_v = train_test_split(X_train_temp, y_train_temp, test_size=0.1, random_state=42)


In [57]:
import tensorflow as tf

tf.config.experimental.list_physical_devices()
# Define your model architecture
model = tf.keras.Sequential([
    # Add an RNN layer
    tf.keras.layers.SimpleRNN(50, activation='relu', return_sequences=True, input_shape=(None, 1)),
    tf.keras.layers.SimpleRNN(50, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
history = model.fit(
    X_tr, 
    y_tr, 
    epochs=10,
    validation_data=(X_v, y_v)
)

Epoch 1/10


2024-03-13 11:53:33.984641: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f1e480282c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-13 11:53:33.984681: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX350, Compute Capability 6.1
2024-03-13 11:53:34.003938: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-13 11:53:34.529691: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1710330814.604808   31128 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8654/8654 [==============================] - 363s 42ms/step - loss: 5.9599 - val_loss: 5.3122
Epoch 2/10
8654/8654 [==============================] - 513s 59ms/step - loss: 5.3049 - val_loss: 5.2879
Epoch 3/10
6478/8654 [=====================>........] - ETA: 1:53 - loss: 5.1961

In [ ]:
model.save("6hrRNN")